# Training a Sentiment Classifier

This is a notebook for [CMU CS11-711 Advanced NLP](http://phontron.com/class/anlp2024/) that trains a sentiment classifier based on data. Specifically, it uses a bag-of-words to extract features, and the structured perceptron algorithm to train the classifier.

It will take in a text `X` and return a `label` of "1" if the sentiment of the text is positive, "-1" if the sentiment of the text is negative, and "0" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the [Stanford Sentiment Treebank](http://nlp.stanford.edu/sentiment/index.html) by running the notebook all the way to end.

## Setup

Setup code, do imports.

In [14]:
!pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [15]:
import random
import tqdm

## Feature Extraction

Feature extraction code, how do we get the features we use in training? By default we just use every word.

In [16]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [17]:
feature_weights = {}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [18]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [19]:
x_train, y_train = read_xy_data('../data/sst-sentiment-text-threeclass/train.txt')
x_dev, y_dev = read_xy_data('../data/sst-sentiment-text-threeclass/dev.txt')

In [20]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


In [21]:
f_x0 = extract_features(x_train[0])

In [22]:
print(f_x0)

{'The': 1.0, 'Rock': 1.0, 'is': 1.0, 'destined': 1.0, 'to': 2.0, 'be': 1.0, 'the': 1.0, '21st': 1.0, 'Century': 1.0, "'s": 2.0, 'new': 1.0, '``': 1.0, 'Conan': 1.0, "''": 1.0, 'and': 1.0, 'that': 1.0, 'he': 1.0, 'going': 1.0, 'make': 1.0, 'a': 1.0, 'splash': 1.0, 'even': 1.0, 'greater': 1.0, 'than': 1.0, 'Arnold': 1.0, 'Schwarzenegger': 1.0, ',': 1.0, 'Jean-Claud': 1.0, 'Van': 1.0, 'Damme': 1.0, 'or': 1.0, 'Steven': 1.0, 'Segal': 1.0, '.': 1.0}


## Inference Code

How we run the classifier.

In [23]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

In [24]:
run_classifier(f_x0)

0

In [36]:
feature_weights ={}

## Training Code

Learn the weights of the classifier.

In [37]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS + 1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            # print(f'The features: {features} with predicted label: {predicted_y} vs true label: {y}')
            for feature in features:
                # print(f'[D] Word Feature is: "{feature}"')
                # print(f'[D] Frequency: {features[feature]}')
                # print(f'[D] Weight before: {feature_weights}')
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]
                # print(f'[D] Weight after: {feature_weights}')
            # break
                

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 296058.73it/s]


In [43]:
feature_weights

{'You': 2.0,
 'can': 0.0,
 'sip': 1.0,
 'your': 1.0,
 'vintage': 4.0,
 'wines': 1.0,
 'and': 2.0,
 'watch': 2.0,
 'Merchant': -1.0,
 'Ivory': 1.0,
 'productions': 1.0,
 ';': 4.0,
 'I': -3.0,
 "'ll": -1.0,
 'settle': -4.0,
 'for': 2.0,
 'a': 2.0,
 'nice': 5.0,
 'cool': 6.0,
 'glass': 1.0,
 'of': 2.0,
 'iced': 1.0,
 'tea': 2.0,
 'Jerry': 0.0,
 'Bruckheimer': 2.0,
 'flick': -2.0,
 'any': -3.0,
 'day': 7.0,
 'the': 0.0,
 'week': -3.0,
 '.': -1.0,
 'In': -4.0,
 'process': 3.0,
 'trimming': -1.0,
 'movie': 3.0,
 'to': -1.0,
 'an': 4.0,
 'expeditious': -1.0,
 '84': 1.0,
 'minutes': -7.0,
 ',': -1.0,
 'director': 1.0,
 'Roger': 2.0,
 'Kumble': -1.0,
 'seems': -8.0,
 'have': -4.0,
 'dumped': -1.0,
 'whole': -5.0,
 'lot': 2.0,
 'plot': -5.0,
 'in': 2.0,
 'favor': -4.0,
 '...': -3.0,
 'outrageous': 1.0,
 'gags': -5.0,
 'The': -1.0,
 'only': -7.0,
 'pain': -1.0,
 'you': 2.0,
 'feel': 0.0,
 'as': -3.0,
 'credits': 1.0,
 'roll': 6.0,
 'is': 0.0,
 'stomach': -3.0,
 'grumbling': -2.0,
 'some': -3.0,
 

## Evaluation Code

How we evaluate the classifier:

In [38]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [39]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [40]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.7469569288389513
Dev/test accuracy: 0.5712988192552225


## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [41]:
def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        y_preds.append(run_classifier(extract_features(x)))
        if y != y_preds[-1]:
            error_ids.append(i)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]
        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}\n')

In [42]:
find_errors(x_dev, y_dev)

It has the ability to offend and put off everyone , but it holds you with its outrageousness .
true label: 0
predicted label: 1

As surreal as a dream and as detailed as a photograph , as visually dexterous as it is at times imaginatively overwhelming .
true label: 1
predicted label: -1

should be seen at the very least for its spasms of absurdist humor .
true label: 0
predicted label: -1

Despite impeccable acting ... and a script that takes some rather unexpected -LRB- even , at times , preposterous -RRB- turns , Love is just too , too precious in the end .
true label: 0
predicted label: -1

Like all abstract art , the film does not make this statement in an easily accessible way , and -- unless prewarned -- it would be very possible for a reasonably intelligent person to sit through its tidal wave of imagery and not get this vision at all .
true label: 0
predicted label: 1

